In [11]:
import os
import json
import requests
import time
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = "https://ai-bcds.openai.azure.com/",
  api_key= "8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
  api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
  model="gpt-4o-mini", # replace with model deployment name.
  instructions="Tu és um perito que trabalha na Fidelidade em investimentos a prazo. Toda a informação que dás é com base nos ficheiros disponibilizados.",
  tools=[{"type":"file_search"}],
  tool_resources={"file_search":{"vector_store_ids":["vs_ftCso8RzHlBSbUwcqVWP3RiG"]},"code_interpreter":{"file_ids":["assistant-WuJZaGApRBgPZGvfPJwTBF","assistant-WQbe2oYNiunshZFamDkAMe","assistant-WA9mo2ooGuCakdtLYe7S5Y","assistant-VmGKDJtKWW8mLr7RsNDHJ4","assistant-VcoN7vmxYPZCBfSTaFsEQ3","assistant-TaHujXgSvATXXH1V5gBTDN","assistant-NMXvUzELtRbwtQMoTXeApo","assistant-LvBZjKCKvtwUvCCTgpfWim","assistant-GWrPgzCPp8si1AH2yKKJBB","assistant-F5nrFbNEjGxEqeSCmkdLAg","assistant-EwB2EpAtrmQVUrtxvg79HB","assistant-BFYazbjKgte4W5u9Jgrs6b","assistant-96mBrzvMJbaTjLYcCEG3S2","assistant-8phYrqvFLjksiThSKqSgkq","assistant-7GHqFyWe29skpMH8Ug4QoZ","assistant-6zJbn35vHos3RRzqJUiVMQ","assistant-5dMu4U3q2VUkjhP4LvhAGb","assistant-4hduKqwh6y7AWcUohbxMC3","assistant-426n1B453C2wqH2AnDc2mh","assistant-3UFjejDd1iMmYhBuj2bgi8"]}},
  temperature=0.44,
  top_p=1
)

In [18]:
# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="O Fidelidade Savings é mais indicado para que perfil de risco?"
)



# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
    pass

SyncCursorPage[Message](data=[Message(id='msg_K4mHxqhBVPPyZEtUJKRpVXNf', assistant_id='asst_oelnbXoPfr1bIszYkMFZPdcY', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=394, file_citation=FileCitation(file_id='assistant-X9pi8pvQhpkfz91qAieknm'), start_index=381, text='【44:3†source】', type='file_citation'), FileCitationAnnotation(end_index=407, file_citation=FileCitation(file_id='assistant-S3DWP1wxGcRLRBTUbYEWqg'), start_index=394, text='【44:9†source】', type='file_citation'), FileCitationAnnotation(end_index=677, file_citation=FileCitation(file_id='assistant-LsShNNEN9Locy2X6QSyiu7'), start_index=664, text='【44:8†source】', type='file_citation'), FileCitationAnnotation(end_index=691, file_citation=FileCitation(file_id='assistant-SZzZsiAGNX82AP5e4mYgqc'), start_index=677, text='【44:12†source】', type='file_citation'), FileCitationAnnotation(end_index=1006, file_citation=FileCitation(file_id='assistant-SZzZsiAGNX82AP5e4mYgqc'

In [19]:
for msg in messages.data:
    if msg.role == "assistant":
        print("Assistant:", msg.content[0].text.value)


Assistant: O produto Fidelidade Savings é mais indicado para diferentes perfis de risco, conforme as opções disponíveis:

1. **Opção Seguro**: Destina-se a clientes com perfil **prudente**. Esta opção oferece uma garantia de 100% do capital investido e um rendimento garantido (variável) ao longo da vigência do contrato. É ideal para aqueles que valorizam a segurança e desejam evitar riscos【44:3†source】【44:9†source】.

2. **Opção Proteção**: Adequada para clientes com perfil **equilibrado**. Esta opção garante 90% do capital investido e oferece um potencial de rendimento indexado a um índice de ações, permitindo um equilíbrio entre segurança e a possibilidade de ganhos【44:8†source】【44:12†source】.

3. **Opção Dinâmico**: Voltada para clientes com perfil **dinâmico**. Não oferece garantia do capital investido e busca maximizar a valorização através de investimentos em ações e obrigações, sendo indicada para aqueles que aceitam um maior nível de risco em troca de potenciais retornos mais al